<a href="https://colab.research.google.com/github/pmontman/TSForecasting/blob/master/experiments/forecastingdata_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Title: Monash Time Series Forecasting Archive

#### Group Member Names : Yash Ramani & Ashishkumar Savani

####  Emails : 200572154@student.georgianc.on.ca & 200564982@student.georgianc.on.ca

### Introduction:
in Forecastingdata, some forecasting and feature calculation tools are implemented in R, but we can access them *transparently* from python using the awesome rpy2 package.
This notebook shows an example for it.


It has two parts. Part 1 is the preparation of the environment so everything
runs directly on python. Part2 is an example of how to use the tools directly from python.

#### AIM :
This project analytically examine the usefulness related to the experiments on a set of publicly available datasets that are used in the time series forecasting research space.
*********************************************************************************************************************
#### Github Repo:
Original Repo:https://github.com/rakshitha123/TSForecasting/tree/master
Contributed Repo: https://github.com/RamaniYash123/MLP_final_term_project/tree/main
*********************************************************************************************************************
#### DESCRIPTION OF PAPER:
In this paper, we present a comprehensive time series forecasting repository which includes publicly available time series datasets from multiple domains, each with a unique frequency, series length, and inclusion of missing values. We also analyze the datasets and figure out similarities and differences between them using feature analysis. In addition, we illustrate the usefulness of a set of baseline forecasting standards across all datasets and eight error metrics.
*********************************************************************************************************************
#### PROBLEM STATEMENT :
* The research only compares 13 baselines, which means that it misses certain newly published works. It would be more informative to compare with more advanced time-series forecasting models.

* Time-series lack a unified metrics for evaluation. This paper raises this problem but leaves the decision to the owners of specific applications.
*********************************************************************************************************************
#### CONTEXT OF THE PROBLEM:
* There are currently no comprehensive time series forecasting benchmarking archives, to the best of our knowledge, that focus on such datasets of related time series to evaluate the performance of such global forecasting algorithms.
*********************************************************************************************************************
#### SOLUTION:
Our aim is to address this limitation by introducing such an archive. In particular,
our paper has the following main contributions.

* The dataset is constructed in a sound way. The experiments are performed reasonably.
* We can see major difference in Median MASE, Mean RMSE and Median RMSE but that might be due to low data size. Apart from this we also added one extra model known as a Seasonal Decomposition of Time Series (STL). To add this model we made changes in experiments/fixed_horizon.R, models/local_univariate_models.R, experiments/rolling_origin.R these files.
* In this the full pipeline is executed for all local and global models using two single function calls.

## Part 1) Preparation of the environment

We load the R extension for jupyter, which interacts nicely with python

In [ ]:
#@title
%load_ext rpy2.ipython

In [ ]:
from warnings import filterwarnings
filterwarnings(action='ignore')

In [ ]:
!git clone https://github.com/RamaniYash123/MLP_final_term_project.git

Cloning into 'MLP_final_term_project'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 42 (delta 6), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (42/42), 230.96 KiB | 4.12 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [ ]:
%cd MLP_final_term_project/
!git pull
%cd ..

/MLP_final_term_project
Already up to date.
/


We now prepare the R environment for processing: installing and loading the required packages

In [ ]:
%%R
install.packages("devtools")
# devtools::install_github("hendersontrent/catch22")
install.packages("tidyverse")
install.packages("tsibble")
install.packages("forecast")
install.packages("tsfeatures")
install.packages("smooth")
# library("Rcatch22")
install.packages("Rcatch22")

(as ‘lib’ is unspecified)










































































	‘/tmp/RtmprMYEqK/downloaded_packages’

(as ‘lib’ is unspecified)







	‘/tmp/RtmprMYEqK/downloaded_packages’

(as ‘lib’ is unspecified)



















	‘/tmp/RtmprMYEqK/downloaded_packages’

(as ‘lib’ is unspecified)
































































	‘/tmp/RtmprMYEqK/downloaded_packages’

(as ‘lib’ is unspecified)







































	‘/tmp/RtmprMYEqK/downloaded_packages’

(as ‘lib’ is unspecified)


































	‘/tmp/RtmprMYEqK/downloaded_packages’

(as ‘lib’ is unspecified)







	‘/tmp/RtmprMYEqK/downloaded_packages’



Install glmnet package for linear models

In [ ]:
%%R
install.packages('glmnet')

(as ‘lib’ is unspecified)





























	‘/tmp/RtmprMYEqK/downloaded_packages’



In [ ]:
%cd ..
! ls MLP_final_term_project/experiments

/
deep_learning_experiments.py  fixed_horizon_functions.R     rolling_origin.R
feature_experiments.R	      fixed_horizon.R
feature_functions.R	      forecastingdata_python.ipynb


In [ ]:
%%R
BASE_DIR <- "MLP_final_term_project"
source("MLP_final_term_project/experiments/feature_functions.R")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


The fixed horizon we need to try line by line to skip errors

In [ ]:
%%R
source("MLP_final_term_project/experiments/fixed_horizon_functions.R")




Attaching package: ‘greybox’



    hm



    spread





Attaching package: ‘Matrix’



    expand, pack, unpack





## Part 2) Using tools directly on Python

Load the function into python environment through rpy2 so
it can be later accessed directly from python code

In [ ]:
import rpy2.robjects as robjects
calculate_features = robjects.r['calculate_features']
do_fixed_horizon_local_forecasting = robjects.r['do_fixed_horizon_local_forecasting']
do_fixed_horizon_global_forecasting = robjects.r['do_fixed_horizon_global_forecasting']


we can now call `calculate_features` directly from python, here we have two examples using both the 'tsfeatures' and the 'catch22' features.
We will first remove the files just in case

Compute the features, we capture the stdout output to remove the verbosiness  of the progress report

In [ ]:
%%capture cap --no-stderr
calculate_features("sample", "sample.tsf", "series_name", "start_timestamp", "tsfeatures")
calculate_features("sample", "sample.tsf", "series_name", "start_timestamp", "catch22")

  method            from
  as.zoo.data.frame zoo 



The functions write the outputs to .csv files in the project directory
structure. We can quickly check if the experiments runs OK, we should see some features in the following files.

In [ ]:
!echo "head of the tsfeatures features"
!ls -r TSForecasting/results/tsfeatures
!head -5 TSForecasting/results/tsfeatures/sample_features.csv
!echo "head of the catch22 features"
!ls -r TSForecasting/results/catch22_features
!head -5 TSForecasting/results/catch22_features/sample_features.csv

head of the tsfeatures features
ls: cannot access 'TSForecasting/results/tsfeatures': No such file or directory
head: cannot open 'TSForecasting/results/tsfeatures/sample_features.csv' for reading: No such file or directory
head of the catch22 features
ls: cannot access 'TSForecasting/results/catch22_features': No such file or directory
head: cannot open 'TSForecasting/results/catch22_features/sample_features.csv' for reading: No such file or directory


Compute the local forecasts.

In [ ]:
do_fixed_horizon_local_forecasting("sample", "ses", "sample.tsf", "series_name", "start_timestamp", 8)

[1] "Started loading sample"
[1] "started Forecasting"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131
[1] 1

<rpy2.rinterface_lib.sexp.NULLType object at 0x7a2172af9d00> [RTYPES.NILSXP]

Try the global model forecasts.

In [ ]:
do_fixed_horizon_global_forecasting("sample", 25, "sample.tsf", "pooled_regression", "series_name", "start_timestamp", 8)

[1] "Started loading sample"
[1] "started Forecasting"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131
[1] 1

<rpy2.rinterface_lib.sexp.NULLType object at 0x7a2172af9d00> [RTYPES.NILSXP]

## Part 3) My contribution:

I take new dataset from https://zenodo.org/communities/forecasting this site. this new  set of for-hire vehicle (FHV) firms' journeys and vehicle numbers are captured by 329 daily time series in this dataset.

In [ ]:
calculate_features("vehicletrips", "vehicletrips.tsf", "series_name", "start_timestamp", "tsfeatures")
calculate_features("vehicletrips", "vehicletrips.tsf", "series_name", "start_timestamp", "catch22")

[1] "Started feature calculation: vehicletrips"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76


  zero-variance series


  zero-variance series


 



[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100


  zero-variance series


  zero-variance series


 



[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131
[1] 132
[1] 133
[1] 134
[1] 135
[1] 136
[1] 137
[1] 138
[1] 139
[1] 140
[1] 141
[1] 142
[1] 143
[1] 144
[1] 145
[1] 146
[1] 147
[1] 148
[1] 149
[1] 150
[1] 151
[1] 152
[1] 153
[1] 154
[1] 155
[1] 156
[1] 157
[1] 158
[1] 159
[1] 160
[1] 161
[1] 162
[1] 163
[1] 164
[1] 165
[1] 166
[1] 167
[1] 168
[1] 169
[1] 170
[1] 171
[1] 172
[1] 173
[1] 174
[1] 175
[1] 176
[1] 177
[1] 178
[1] 179
[1] 180
[1] 181
[1] 182
[1] 183
[1] 184
[1] 185
[1] 186
[1] 187
[1] 188
[1] 189
[1] 190
[1] 191
[1] 192
[1] 193
[1] 194
[1] 195
[1] 196
[1] 197
[1] 198
[1] 199
[1] 200
[1] 201
[1] 202
[1] 203
[1] 204
[1] 205
[1] 206
[1] 207
[1] 208
[1] 209
[1] 210
[1] 211
[1] 212
[1] 213
[1] 214
[1] 215
[1] 216
[1] 217
[1] 218
[1] 219
[1] 220
[1] 221
[1] 222
[1] 223
[1] 224
[1] 225


...

  zero-variance series


  zero-variance series


 



[1] 299
[1] 300


  zero-variance series


  zero-variance series


 



[1] 301
[1] 302
[1] 303
[1] 304
[1] 305
[1] 306
[1] 307
[1] 308
[1] 309
[1] 310
[1] 311
[1] 312
[1] 313
[1] 314
[1] 315
[1] 316
[1] 317
[1] 318
[1] 319
[1] 320
[1] 321
[1] 322
[1] 323
[1] 324
[1] 325
[1] 326
[1] 327
[1] 328
[1] 329
[1] "Started feature calculation: vehicletrips"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 

<rpy2.rinterface_lib.sexp.NULLType object at 0x7a2172af9d00> [RTYPES.NILSXP]

In [ ]:
do_fixed_horizon_local_forecasting("vehicletrips", "ses", "vehicletrips.tsf", "series_name", "start_timestamp", 8)


[1] "Started loading vehicletrips"
[1] "started Forecasting"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131

<rpy2.rinterface_lib.sexp.NULLType object at 0x7a2172af9d00> [RTYPES.NILSXP]

In [ ]:
do_fixed_horizon_global_forecasting("vehicletrips", 25, "vehicletrips.tsf", "pooled_regression", "series_name", "start_timestamp", 8)

[1] "Started loading vehicletrips"
[1] "started Forecasting"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131

<rpy2.rinterface_lib.sexp.NULLType object at 0x7a2172af9d00> [RTYPES.NILSXP]

We can see major difference in Median MASE, Mean RMSE and Median RMSE but that might be due to low data size.
 Apart from this I also add one extra model known as a Seasonal Decomposition of Time Series (STL). To add this model I made changes in experiments/fixed_horizon.R, models/local_univariate_models.R, experiments/rolling_origin.R these files. In this  the full pipeline is executed for all local and global models using two single function calls.

# Citation:
@InProceedings{godahewa2021monash,
    author = "Godahewa, Rakshitha and Bergmeir, Christoph and Webb, Geoffrey I. and Hyndman, Rob J. and Montero-Manso, Pablo",
    title = "Monash Time Series Forecasting Archive",
    booktitle = "Neural Information Processing Systems Track on Datasets and Benchmarks",
    year = "2021"
}

# References:

[1]: Wang, E., Cook, D., Hyndman, R. J. (2020). A new tidy data structure to support exploration and modeling of temporal data. Journal of Computational and Graphical Statistics. doi:10.1080/10618600.2019.1695624.

[2]: R Core Team (2018). foreign: Read Data Stored by 'Minitab', 'S', 'SAS', 'SPSS', 'Stata', 'Systat', 'Weka', 'dBase', .... R package version 0.8-71. https://CRAN.R-project.org/package=foreign